<a href="https://colab.research.google.com/github/4rldur0/whyfi/blob/xeoyeon/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 초기 설정

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/RAG

In [ ]:
!pip install transformers chromadb langchain langchain_community langchain-chroma

# 2. data 불러오기

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/RAG/dataset/cleaned_word_dict.csv'
data = pd.read_csv(file_path)

## pdf 가져와서 chunk하기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_name = # 읽어오려는 파일 경로

loader = PyPDFLoader(file_name)
pages = loader.load()
text = ""
for page in pages:
    sub = page.page_content
    text += sub

In [ ]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# token size 기준으로 contents split
tokenizer = AutoTokenizer.from_pretrained(ENCODER)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer,
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            separator="\n" # default: "\n\n"
        )


documents=[] # split 한 문서들을 담기 위한 array

split_conts = text_splitter.split_text(text)
for chunk_idx, split_cont in enumerate(split_conts):
    documents.append(Document(
        page_content=split_cont,
        metadata={
            "file_name": file_name,
        },
        id=chunk_idx,
    ))
    idx+=1

vectorstore.add_documents(documents)

## website 문서 로딩하기

In [ ]:
# Load documents from web
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader([
    "링크 삽입"
    ]
)

data = web_loader.load()

In [ ]:
##Split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

all_splits = text_splitter.split_documents(data)

all_splits[0]
출처: https://rfriend.tistory.com/832 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]

# 3. chromaDB에 데이터 저장하기

In [ ]:
!pip install langchain-huggingface

In [ ]:
#기존 임베딩 결과만 가져오기
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정 (1000개로 확장)
)


In [ ]:
# 기존 컬렉션 삭제
vectorstore._client.delete_collection("Chroma_Collection")

# 새로운 컬렉션 생성
vectorstore = Chroma(
    collection_name="Chroma_Collection",
    embedding_function=embedding_func,
    persist_directory="/content/drive/MyDrive/Colab Notebooks/RAG",
    collection_metadata={"max_size": 1000}
)


In [ ]:
from langchain.docstore.document import Document # 데이터의 각 row를 document 객체로 변환하여 저장하기 위함

# vectorDB에 data 추가하는 함수
def add_data_to_vectorstore(data, vectorstore):
  for index, row in data.iterrows(): # row라는 변수에 각 행을 반복적으로 가져옴.
    text = row.get("Content","")
    metadata = row.to_dict() #행 전체를 딕셔너리 형태로 변환
    metadata["source"] = metadata.get("source", f"row_{index}")   # source 필드 추가 (기본값으로 행 번호를 사용하거나 특정 열에서 가져오기)
    document = Document(page_content=text, metadata=metadata) #cocument 객체를 생성
    vectorstore.add_texts([document.page_content],[document.metadata]) # db에 데이터 추가

add_data_to_vectorstore(data, vectorstore)

# 4. Retriever 및 프롬프트 설정

In [ ]:
# Define a retriever to search in the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) #검색 시 가장 관련성 높은 3개의 문서를 반환하라는 뜻

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# system_template="""
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

# You MUST answer in Korean.

# Question: {question}
# Context: {context}
# Answer:
# """
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어나 개념을 최대 3개까지 추천하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

# 5. gemini api 설정 및 chain 구현하기

In [ ]:
!pip install langchain-google-vertexai

In [ ]:
PROJECT_ID = "your_project_id"
REGION = "your_region"

from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# Chain 사용 예시
question = "가동률이란?"
result = chain({"question": question})

print("Answer:", result["answer"])
print("Sources:", result["source_documents"])

# 6. streamlit 으로 간단하게 웹 ui 구현하기
1. [ngrok](https://dashboard.ngrok.com/) 에 접속 후 회원가입
2. 로그인 후 뜨는 authtoken 번호를 아래의 코드에 붙여넣기


In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py

import streamlit as st
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import re

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정 (1000개로 확장)
)

#retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

#초기화
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어를 추천하세요.

단, 반드시 한국어로 답변해야 하며, 전체 Answer의 길이가 8줄을 넘어가지 않도록 하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

#gemini
PROJECT_ID = "your_project_id"
REGION = "your_region"

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

#streamlit 설계
st.set_page_config(page_title="금융용어알리미 : WhyFi")

st.markdown(
    """
    <h1 style='color: white; text-align: center;'>금융용어알리미 : WhyFi</h1>
    <h2 style='color: gray; text-align: center; font-size: 20px;'>모르는 금융 용어? 이제 쉽게 찾아보세요!</h2>
    <br>
    """,
    unsafe_allow_html=True,
)


user_question = st.text_input("금융 용어를 검색해보세요.", value="", placeholder="예: 가동률이란?")
if user_question:
    with st.spinner("답변을 생성 중입니다..."):
        result = chain({"question": user_question})

    # 결과 출력
    st.markdown("### 📖 답변 📖")

    # 텍스트에서 헤더(##)를 분리하고 스타일 적용
    answer_text = result["answer"]

    # 헤더와 본문 분리
    header_match = re.match(r"^##\s*(.*)", answer_text)
    if header_match:
        header = header_match.group(1)  # 헤더 부분
        body = re.sub(r"^##\s*.*\n*", "", answer_text, flags=re.MULTILINE)  # 나머지 텍스트
    else:
        header = ""
        body = answer_text

    # 헤더와 본문 각각 스타일링
    st.markdown(
        f"<div style='font-size:20px; font-weight:bold; line-height:1.8;'>{header}</div>",
        unsafe_allow_html=True,
    )
    st.markdown(
        f"<div style='font-size:16px; line-height:1.6;'>{body}</div>",
        unsafe_allow_html=True,
    )

    st.markdown("### 📚 참고 출처 📚")
    for doc in result["source_documents"]:
        st.markdown(
            f"<div style='font-size:14px; line-height:1.4;'>- {doc.metadata['source']}</div>",
            unsafe_allow_html=True,
        )

#         # 결과 출력
# st.markdown("#### 📖 답변:")
# st.markdown(f"<div style='font-size:16px; line-height:1.6;'>{result['answer']}</div>", unsafe_allow_html=True)

# st.markdown("#### 📚 참고 출처:")
# for doc in result["source_documents"]:
#     st.markdown(f"<div style='font-size:14px; line-height:1.4;'>- {doc.metadata['source']}</div>", unsafe_allow_html=True)


In [ ]:
!ls

In [ ]:
#개인 토큰 번호 입력
!ngrok authtoken [개인 토큰 번호]

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&

In [ ]:
publ_url = ngrok.connect(addr="8501")

In [ ]:
publ_url

In [ ]:
!ps

In [ ]:
!kill 13177 17180 12664

In [ ]:
ngrok.kill()